In [1]:
import json

In [3]:
with open("labeled_tz.json") as file:
    labeled_all = json.load(file)

In [5]:
labeled_all['0']

{'id': 0,
 'task_path': 'images/000031_000000_img_bbox_bbox_a64a23bc-7aa9-11eb-a429-0242c0a88002.jpg',
 'labeled': ['raw_text'],
 'data': {'original_document': '1614615445_669.zip',
  '_metadata': {'paragraph_type': 'raw_text',
   'predicted_classes': None,
   'page_id': 2,
   'line_id': 6},
  '_hierarchy_level': {'paragraph_type': 'raw_text',
   'level_1': None,
   'can_be_multiline': False,
   'level_2': None},
  '_annotations': [{'name': 'size', 'value': '12', 'start': 0, 'end': 19}],
  '_line': 'прочность грузов |\n',
  '_uid': 'bbox_a64a23bc-7aa9-11eb-a429-0242c0a88002',
  'color': '#f0f',
  'bbox': {'line_num': 6,
   'original_image': 'img_1614615445_669.pdf_000002.png',
   '_uid': 'bbox_a64a23bc-7aa9-11eb-a429-0242c0a88002',
   'label': None,
   'page_num': 2,
   'text': 'прочность грузов |\n',
   'uid': 'bbox_a64a23bc-7aa9-11eb-a429-0242c0a88002',
   'bbox': {'height': 33,
    'x_top_left': 251,
    'width': 1317,
    'y_top_left': 380}}},
 'default_label': 'raw_text'}

In [7]:
labeled = {}
for key, value in labeled_all.items():
    if value["data"]["original_document"].endswith(".docx"):
        labeled[key] = value

In [8]:
len(labeled_all)

6647

In [9]:
len(labeled)

1393

In [13]:
list(labeled.items())[0]

('185',
 {'id': 185,
  'task_path': 'images/000032_000000_img_bbox_28e466087d8f9f2c796a7f3674bf97d5_afe029eb67c4335e35a37f3bdc7a7a3f.jpg',
  'labeled': ['title'],
  'data': {'original_document': '1614615210_93.docx',
   '_metadata': {'paragraph_type': 'root',
    'predicted_classes': None,
    'page_id': 0,
    'line_id': 2},
   '_hierarchy_level': {'paragraph_type': 'root',
    'level_1': 0,
    'can_be_multiline': True,
    'level_2': 0},
   '_annotations': [{'name': 'indentation',
     'value': '0',
     'start': 0,
     'end': 13},
    {'name': 'alignment', 'value': 'center', 'start': 0, 'end': 13},
    {'name': 'size', 'value': '12.0', 'start': 0, 'end': 13},
    {'name': 'style', 'value': 'normal', 'start': 0, 'end': 13}],
   '_line': ' Приложение 1\n',
   '_uid': '28e466087d8f9f2c796a7f3674bf97d5_afe029eb67c4335e35a37f3bdc7a7a3f',
   'color': '#f00'},
  'default_label': 'title'})

In [14]:
with open('labeled_tz.json', 'w') as outfile:
    json.dump(labeled, outfile)